In [1]:
!pip install findspark


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
# 1. Inicializar Spark en Jupyter
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("StructuredStreamingNotebook") \
    .master("local[*]") \
    .getOrCreate()

25/05/21 00:16:21 WARN Utils: Your hostname, codespaces-367222 resolves to a loopback address: 127.0.0.1; using 10.0.2.67 instead (on interface eth0)
25/05/21 00:16:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/21 00:16:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 2. Crear un DataFrame de streaming simulado (fuente = rate)
df = spark.readStream \
    .format("rate") \
    .option("rowsPerSecond", 1) \
    .load()

df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)



In [4]:
# 3. Escribir el streaming a memoria (queryName = rate_table)
query = df.writeStream \
    .format("memory") \
    .queryName("rate_table") \
    .outputMode("append") \
    .start()

25/05/21 00:19:39 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4200d774-b32f-4c71-abc6-429352fc3752. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/21 00:19:39 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
# 4. Esperar unos segundos para acumular datos
import time
time.sleep(5)

In [9]:
# 5. Consultar los datos que se han acumulado en memoria
spark.sql("SELECT * FROM rate_table").show()

+--------------------+-----+
|           timestamp|value|
+--------------------+-----+
|2025-05-21 00:19:...|    0|
|2025-05-21 00:19:...|    1|
|2025-05-21 00:19:...|    2|
|2025-05-21 00:19:...|    3|
|2025-05-21 00:19:...|    4|
|2025-05-21 00:19:...|    5|
|2025-05-21 00:19:...|    6|
|2025-05-21 00:19:...|    7|
|2025-05-21 00:19:...|    8|
|2025-05-21 00:19:...|    9|
|2025-05-21 00:19:...|   10|
|2025-05-21 00:19:...|   11|
|2025-05-21 00:19:...|   12|
|2025-05-21 00:19:...|   13|
|2025-05-21 00:19:...|   14|
|2025-05-21 00:19:...|   15|
|2025-05-21 00:19:...|   16|
|2025-05-21 00:19:...|   17|
|2025-05-21 00:19:...|   18|
|2025-05-21 00:19:...|   19|
+--------------------+-----+
only showing top 20 rows



In [10]:
# 6. (Opcional) Detener la consulta
query.stop()

In [11]:
!pyspark --version

25/05/21 00:22:01 WARN Utils: Your hostname, codespaces-367222 resolves to a loopback address: 127.0.0.1; using 10.0.2.67 instead (on interface eth0)
25/05/21 00:22:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.5
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 11.0.14.1
Branch HEAD
Compiled by user ubuntu on 2025-02-23T20:30:46Z
Revision 7c29c664cdc9321205a98a14858aaf8daaa19db2
Url https://github.com/apache/spark
Type --help for more information.
